In [1]:
import numpy as np
import cv2 as cv
import sys

In [14]:
# 사전 학습 모델을 읽어 YOLO 구성
def construct_yolo_v3():
    f = open('./files/coco_names.txt', 'r')
    class_names = [line.strip() for line in f.readlines()]
    
    # cfg = 컨피그 파일
    model = cv.dnn.readNet('./files/yolov3.weights', './files/yolov3.cfg')
    layer_names = model.getLayerNames()
    out_layers = [layer_names[i-1] for i in model.getUnconnectedOutLayers()]
    
    return model, out_layers, class_names

In [15]:
# YOLO 모델로 물체를 검출
def yolo_detect(img, yolo_model, out_layers):
    height, width = img.shape[0], img.shape[1]
    # print(img.shape)
    test_img = cv.dnn.blobFromImage(img, 1.0/256, (448, 448), (0,0,0), swapRB=True)
    
    yolo_model.setInput(test_img)
    output3 = yolo_model.forward(out_layers)
    
    box, conf, id = [], [], [] # 박스, 신뢰도, 분류 번호
    for output in output3:
        for vec85 in output:
            scores = vec85[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 : # 신뢰도가 50% 이상인 경우만 취함
                centerx, centery = int(vec85[0]*width), int(vec85[1]*height)
                w, h = int(vec85[2]*width), int(vec85[3]*height)
                x, y = int(centerx - w/2), int(centery - h/2)
                box.append([x, y, x+w, y+h])
                conf.append(float(confidence))
                id.append(class_id)
                
    ind = cv.dnn.NMSBoxes(box, conf, 0.5, 0.4)
    objects = [box[i] + [conf[i]] + [id[i]] for i in range(len(box)) if i in ind]
    return objects

In [17]:
model, out_layers, class_names = construct_yolo_v3() # 모델 생성
colors = np.random.uniform(0, 255, size=(len(class_names), 3)) # 부류 마다 색깔

img = cv.imread('./imgs/soccer.jpg')
if img is None : sys.exit('파일이 없습니다.')

res = yolo_detect(img, model, out_layers)

for i in range(len(res)):
    x1, y1, x2, y2, confidence, id = res[i]
    text = str(class_names[id]) + '%.3f' %confidence
    cv.rectangle(img,(x1, y1), (x2, y2), colors[id], 2)
    cv.putText(img, text, (x1, y1+30), cv.FONT_HERSHEY_PLAIN, 1.5, colors[id], 2)
    
cv.imshow("Object detection by YOLO v.3", img)

cv.waitKey()
cv.destroyAllWindows()

In [18]:
model,out_layers,class_names=construct_yolo_v3()		# YOLO 모델 생성
colors=np.random.uniform(0,255,size=(len(class_names),3))	# 부류마다 색깔

cap=cv.VideoCapture(0,cv.CAP_DSHOW)
if not cap.isOpened(): sys.exit('카메라 연결 실패')

while True:
    ret,frame=cap.read()
    if not ret: sys.exit('프레임 획득에 실패하여 루프를 나갑니다.')
        
    res=yolo_detect(frame,model,out_layers)   
 
    for i in range(len(res)):
        x1,y1,x2,y2,confidence,id=res[i]
        text=str(class_names[id])+'%.3f'%confidence
        cv.rectangle(frame,(x1,y1),(x2,y2),colors[id],2)
        cv.putText(frame,text,(x1,y1+30),cv.FONT_HERSHEY_PLAIN,1.5,colors[id],2)
    
    cv.imshow("Object detection from video by YOLO v.3",frame)
    
    key=cv.waitKey(1) 
    if key==ord('q'): break 
    
cap.release()		# 카메라와 연결을 끊음
cv.destroyAllWindows()

In [19]:
model,out_layers,class_names=construct_yolo_v3()		# YOLO 모델 생성
colors=np.random.uniform(0,255,size=(len(class_names),3))	# 부류마다 색깔

cap=cv.VideoCapture(0,cv.CAP_DSHOW)
if not cap.isOpened(): sys.exit('카메라 연결 실패')

import time

start=time.time()
n_frame=0
while True:
    ret,frame=cap.read()
    if not ret: sys.exit('프레임 획득에 실패하여 루프를 나갑니다.')
        
    res=yolo_detect(frame,model,out_layers)   
 
    for i in range(len(res)):
        x1,y1,x2,y2,confidence,id=res[i]
        text=str(class_names[id])+'%.3f'%confidence
        cv.rectangle(frame,(x1,y1),(x2,y2),colors[id],2)
        cv.putText(frame,text,(x1,y1+30),cv.FONT_HERSHEY_PLAIN,1.5,colors[id],2)
    
    cv.imshow("Object detection from video by YOLO v.3",frame)
    n_frame+=1
    
    key=cv.waitKey(1) 
    if key==ord('q'): break 

end=time.time()
print('처리한 프레임 수=',n_frame,', 경과 시간=',end-start,'\n초당 프레임 수=',n_frame/(end-start))

cap.release()		# 카메라와 연결을 끊음
cv.destroyAllWindows()

처리한 프레임 수= 66 , 경과 시간= 21.97389817237854 
초당 프레임 수= 3.0035635681138637


### pixellib

In [2]:
from pixellib.semantic import semantic_segmentation
import cv2 as cv

In [21]:
seg = semantic_segmentation()
seg.load_ade20k_model('./files/deeplabv3_xception65_ade20k.h5')

img_fname = './imgs/busy_street.jpg'
seg.segmentAsAde20k(img_fname, output_image_name='./imgs/image_new.jpg')
info1, img_segmented1 = seg.segmentAsAde20k(img_fname)
info2, img_segmented2 = seg.segmentAsAde20k(img_fname, overlay=True)

cv.imshow('Image original', cv.imread(img_fname))
cv.imshow('Image segmention', img_segmented1)
cv.imshow('Image segmention overlayed', img_segmented2)

cv.waitKey()
cv.destroyAllWindows()

Processed Image saved successfuly in your current working directory.


In [ ]:
cap=cv.VideoCapture(0)

seg_video=semantic_segmentation()
seg_video.load_ade20k_model('./files/deeplabv3_xception65_ade20k.h5')

seg_video.process_camera_ade20k(cap,overlay=True,frames_per_second=2,output_video_name='output_video.mp4',show_frames=True,frame_name='Pixellib')

cap.release()
cv.waitKey()
cv.destroyAllWindows()

KeyboardInterrupt: 

: 

In [6]:
from ultralytics import YOLO
import cv2

model = YOLO('./files/yolov8x-seg.pt')
img = cv2.imread('./imgs/busy_street.jpg')
results = model(img)

results[0].show()


0: 448x640 13 persons, 7 bicycles, 3 cars, 2 traffic lights, 8 backpacks, 3 handbags, 30.1ms
Speed: 1.3ms preprocess, 30.1ms inference, 2.8ms postprocess per image at shape (1, 3, 448, 640)


### pixellib 활용한 배경 바꾸기

In [4]:
import cv2 as cv
import numpy as np
from PyQt5.QtWidgets import *
import sys
from pixellib.tune_bg import alter_bg

ImportError: cannot import name 'BatchNormalization' from 'tensorflow.python.keras.layers' (c:\Users\main\miniconda3\envs\dl\Lib\site-packages\tensorflow\python\keras\layers\__init__.py)

In [ ]:
class VideoSpecialEffect(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('배경을 내 맘대로')
        self.setGeometry(200, 200, 400, 100)
        
        VideoButton = QPushButton('배경 내 맘대로 켜기', self)
        self.pickCombo = QComboBox(self)
        self.pickCombo.addItems(['원래 영상', '흐릿(조금)', '흐릿(중간)', '흐릿(많이)',
                                 '빨강', '녹색', '파랑'])
        quitButton = QPushButton('나가기', self)
        
        VideoButton.setGeometry(10, 10, 140, 30)
        self.pickCombo.setGeometry(150, 10, 110, 30)
        quitButton.setGeometry(280, 10, 100, 30)
        
        VideoButton.clicked.connect(self.videoSpecialEffectFunction)
        quitButton.clicked.connect(self.quitFunction)
        
    def videoSpecialEffectFunction(self):
        self.cap = cv.VideoCapture(0, cv.CAP_DSHOW)
        if not self.cap.isOpened(): sys.exit('카메라 연결 실패')
        
        while True:
            ret,frame=self.cap.read()  
            if not ret: break

            pick_effect=self.pickCombo.currentIndex()        
            if pick_effect==0:
                special_img=frame
            elif pick_effect==1:
                special_img=change_bg.blur_frame(frame,low=True,detect='person')
            elif pick_effect==2:
                special_img=change_bg.blur_frame(frame,moderate=True,detect='person')
            elif pick_effect==3:    
                special_img=change_bg.blur_frame(frame,extreme=True,detect='person')
            elif pick_effect==4:    
                special_img=change_bg.color_frame(frame,colors=(255,0,0),detect='person')
            elif pick_effect==5:    
                special_img=change_bg.color_frame(frame,colors=(0,255,0),detect='person')
            elif pick_effect==6:    
                special_img=change_bg.color_frame(frame,colors=(0,0,255),detect='person')
                
            cv.imshow('Special effect',special_img)              
            cv.waitKey(1) 
                                        
    def quitFunction(self):
        self.cap.release()
        cv.destroyAllWindows()        
        self.close()

change_bg=alter_bg(model_type="pb")
change_bg.load_pascalvoc_model('xception_pascalvoc.pb')
                
app=QApplication(sys.argv) 
win=VideoSpecialEffect() 
win.show()
app.exec_()